# Components and mixtures creation


This notebook has the purpose of showing examples of the attributues of components and mixtures objects.


In [1]:
import numpy as np
from phasepy import mixture, component

Critical information (Tc, Pc, Zc, Vc), acentric factor (w), Antoine Coefficients (Ant) and group contribution (GC) members can be given when a component is created.

Phasepy is setted to working with temperature in Kelvin, pressure in bar and volumes in cm3/mol.

In [2]:
water = component(name = 'water', Tc = 647.13, Pc = 220.55, Zc = 0.229, Vc = 55.948, w = 0.344861,
                Ant =  [  11.64785144, 3797.41566067,  -46.77830444],
                GC = {'H2O':1})

ethanol = component(name = 'ethanol', Tc = 514.0, Pc = 61.37, Zc = 0.241, Vc = 168.0, w = 0.643558,
                Ant = [  11.61809279, 3423.0259436 ,  -56.48094263],
                GC = {'CH3':1, 'CH2':1,'OH(P)':1})

print('Water saturation pressure at T = 373K is ', water.psat(T = 373), ' bar')
print('Water liquid volume at T = 310K is ', water.vlrackett(T = 310), ' cm3/mol')
print('Ethanol saturation pressure at T = 373K is ', ethanol.psat(T = 373), ' bar')
print('Ethanol liquid volume at T = 310K is ', ethanol.vlrackett(T = 310), ' cm3/mol')

Water saturation pressure at T = 373K is  1.0072796747419537  bar
Water liquid volume at T = 310K is  16.46025809309672  cm3/mol
Ethanol saturation pressure at T = 373K is  2.233335305328437  bar
Ethanol liquid volume at T = 310K is  56.32856995891473  cm3/mol


Mixtures can be created from components. Also, same methods are available from them.

In [3]:
mix = mixture(ethanol, water)
mix.names
['ethanol', 'water']
mix.nc
#2
mix.psat(T = 373)
#array([2.23333531, 1.00727967])
mix.vlrackett(T = 310)
#array([56.32856996, 16.46025809])

array([56.32856996, 16.46025809])

From mixtures models can be created, the simpliest model is to use a virial expansion for vapor phase and a activity coefficient model for liquid phase. In order to that you need ```virialgama``` function that requires a mixture, virial correlation and activity coefficient model.

In [4]:
from phasepy import virialgama, Abbott, nrtl
alpha = np.array([[0.       , 0.5597628],
       [0.5597628, 0.       ]])
g = np.array([[  0.       , -57.6880881],
       [668.682368 ,   0.       ]])
g1 = np.array([[ 0.        ,  0.46909821],
       [-0.37982045,  0.        ]])
#g are energy parameters of nrtl model in K.
#A = g + g1/T 

#Adding activity model parameters
mix.NRTL(alpha, g, g1)

#Creating Model
model = virialgama(mix, virialmodel = Abbott, actmodel = nrtl )
#Model parameters are saved in actmodelp attribute
parameters = model.actmodelp
np.all(parameters[0] == alpha), np.all(parameters[1] == g), np.all(parameters[2] == g1)

(True, True, True)

In [5]:
#rk is the Redlich Kister Expansion used for Ge/RT
from phasepy import rk
C0 = np.array([ 1.20945699, -0.62209997,  3.18919339])
C1 = np.array([  -13.271128,   101.837857, -1100.29221 ])

#Parameters are calculated as C = C0 + C1/T
mix.rk(C0, C1)
#Creating model
model = virialgama(mix, actmodel = rk )

#Reading parameters
parameters = model.actmodelp
np.all(parameters[0] == C0), np.all(parameters[1] == C1)
parameters[2]

array([[0, 1]])

It is also possible to work in a predictive way, using [Dortmund-Modified UNIFAC](http://www.ddbst.com/PublishedParametersUNIFACDO.html) activity coefficient model.

In [6]:
from phasepy import unifac, ideal_gas
#Reading Dortmund-UNIFAC database
mix.unifac()
#Creating Model
model = virialgama(mix, virialmodel = ideal_gas, actmodel = unifac)

Cubic EOS are also available for phase equilibria and interfacial properties calculations.

In [25]:
from phasepy import preos
#preos refers to Peng Robinson EOS

pr = preos(ethanol)
#computes saturation pressure
pr.psat(T = 350.)
#(array([0.98800647]), array([66.75754804]), array([28799.31921623]))

#liquid density
pr.density(T = 350, P = 1., state = 'L')
#0.01497960198094922

#vapor density
pr.density(T = 350, P = 1., state = 'V')
#3.515440899573752e-05

3.515440899573752e-05

A volume translation can be considered from any of the cubic eos. The attribute ```c``` has to be supplied to the pure component and the option ```volume_translation``` has to be set to ```True```.

The volume translation doesn't change equilibrium and tries to improve the behavior of liquid density predicted by the EoS.

In [28]:
ethanol = component(name = 'ethanol', Tc = 514.0, Pc = 61.37, Zc = 0.241, Vc = 168.0, w = 0.643558,
                c = 5.35490936, Ant = [  11.61809279, 3423.0259436 ,  -56.48094263],
                GC = {'CH3':1, 'CH2':1,'OH(P)':1})

pr = preos(ethanol, volume_translation = True)
#computes saturation pressure
pr.psat(T = 350.)
#(array([0.98800647]), array([61.40263868]), array([28793.96430687]))

#liquid density
pr.density(T = 350, P = 1., state = 'L')
#0.01628597159790686

#vapor density
pr.density(T = 350, P = 1., state = 'V')
#3.5161028012629526e-05

3.5161028012629526e-05

In [29]:
#Quadratic and MHV mixrule were programmed

Kij = np.array([[0, -0.11], [-0.11, 0]])
mix.kij_cubic(Kij)
pr = preos(mix, mixrule = 'qmr')

alpha = np.array([[0.       , 0.5597628],
       [0.5597628, 0.       ]])
g = np.array([[  0.       , -57.6880881],
       [668.682368 ,   0.       ]])
g1 = np.array([[ 0.        ,  0.46909821],
       [-0.37982045,  0.        ]])

#Adding activity model parameters
mix.NRTL(alpha, g, g1)
pr = preos(mix, mixrule = 'mhv_nrtl')

#reading UNIFAC database
mix.unifac()
pr = preos(mix, mixrule = 'mhv_unifac')

C0 = np.array([ 1.20945699, -0.62209997,  3.18919339])
C1 = np.array([  -13.271128,   101.837857, -1100.29221 ])
#Parameters are calculated as C = C0 + C1/T
mix.rk(C0, C1)
pr = preos(mix, mixrule = 'mhv_rk')

For further info about you can check official documentation or just write:

```function?```